#CHAT BOT PROJECT:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Important Libraries:

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D,Bidirectional,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [4]:
with open("/content/drive/MyDrive/PROJECTS/CHATBOT PROJECT/chatbot dataset.json") as file:
  data = json.load(file)

In [5]:
data

{'intents': [{'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"],
   'tag': 'thanks'},
  {'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ["I.m Krishna's chatbot, your bot assistant",
    "I'm, an Artificial Intelligent bot"],
   'tag': 'about'},
  {'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Krishna.',
    "I'm krishnya!",
    'Just call me as kishu'],
   'tag': 'name'},
  {'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help?',
   

In [6]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])
  if intent['tag'] not in labels:
    labels.append(intent['tag'])

len(labels)

8

In [7]:
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'how to open a new account',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service']

In [8]:
training_labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'about',
 'about',
 'about',
 'name',
 'name',
 'name',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'complaint',
 'complaint',
 'complaint']

In [9]:
labels

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'createaccount',
 'complaint']

In [10]:
responses

[['Hello', 'Hi', 'Hi there'],
 ['See you later', 'Have a nice day', 'Bye! Come back again'],
 ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"],
 ["I.m Krishna's chatbot, your bot assistant",
  "I'm, an Artificial Intelligent bot"],
 ['You can call me Krishna.', "I'm krishnya!", 'Just call me as kishu'],
 ['Tell me how can assist you',
  'Tell me your problem to assist you',
  'Yes Sure, How can I support you'],
 ['You can just easily create a new account from our web site',
  'Just go to our web site and follow the guidelines to create a new account'],
 ['Please provide us your complaint in order to assist you',
  'Please mention your complaint, we will reach you and sorry for any inconvenience caused']]

In [11]:
label_encoder = LabelEncoder()
training_labels = label_encoder.fit_transform(training_labels)

In [12]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [17]:

embedding_dim = 30
max_len = 20
oov_token = "<OOV>"

In [18]:
tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word2index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [19]:
sequences

[[29],
 [30],
 [10, 31, 18],
 [32],
 [33],
 [34],
 [35, 2, 36],
 [37],
 [19],
 [38, 2],
 [39, 40],
 [19, 11, 41, 8],
 [20, 12, 2],
 [9, 12, 2],
 [20, 2, 12],
 [9, 10, 21, 22],
 [9, 42, 4, 43, 2],
 [44, 21, 22],
 [45, 2, 8, 5],
 [46, 5, 3, 47, 23],
 [13, 2, 8],
 [9, 13, 2, 48, 11, 5],
 [4, 14, 3, 24],
 [4, 14, 3, 8],
 [24, 5, 23],
 [4, 14, 6, 15, 3, 16, 7],
 [25, 6, 26, 3, 16, 7],
 [4, 27, 6, 15, 28, 7],
 [13, 2, 15, 28, 7, 11, 5],
 [25, 6, 26, 3, 16, 7],
 [49, 3, 17],
 [4, 27, 6, 50, 3, 17],
 [18, 10, 3, 17, 51, 3, 52]]

In [20]:
vocab_size = len(word2index)+1

In [21]:
word2index

{'<OOV>': 1,
 'a': 3,
 'about': 51,
 'account': 7,
 'an': 28,
 'anyone': 31,
 'are': 12,
 'bye': 34,
 'call': 43,
 'can': 13,
 'complaint': 17,
 'could': 45,
 'create': 15,
 'do': 48,
 'for': 11,
 'give': 46,
 'goodbye': 37,
 'hand': 47,
 'have': 49,
 'hay': 33,
 'hello': 32,
 'help': 8,
 'helpful': 40,
 'hey': 30,
 'hi': 29,
 'how': 25,
 'i': 4,
 'is': 10,
 'later': 36,
 'me': 5,
 'name': 22,
 'need': 14,
 'new': 16,
 'open': 26,
 'please': 23,
 'raise': 50,
 'see': 35,
 'service': 52,
 'should': 42,
 'support': 24,
 'thank': 38,
 'thanks': 19,
 "that's": 39,
 'the': 41,
 'there': 18,
 'to': 6,
 'want': 27,
 'what': 9,
 'whats': 44,
 'who': 20,
 'you': 2,
 'your': 21}

In [22]:
sequences

[[29],
 [30],
 [10, 31, 18],
 [32],
 [33],
 [34],
 [35, 2, 36],
 [37],
 [19],
 [38, 2],
 [39, 40],
 [19, 11, 41, 8],
 [20, 12, 2],
 [9, 12, 2],
 [20, 2, 12],
 [9, 10, 21, 22],
 [9, 42, 4, 43, 2],
 [44, 21, 22],
 [45, 2, 8, 5],
 [46, 5, 3, 47, 23],
 [13, 2, 8],
 [9, 13, 2, 48, 11, 5],
 [4, 14, 3, 24],
 [4, 14, 3, 8],
 [24, 5, 23],
 [4, 14, 6, 15, 3, 16, 7],
 [25, 6, 26, 3, 16, 7],
 [4, 27, 6, 15, 28, 7],
 [13, 2, 15, 28, 7, 11, 5],
 [25, 6, 26, 3, 16, 7],
 [49, 3, 17],
 [4, 27, 6, 50, 3, 17],
 [18, 10, 3, 17, 51, 3, 52]]

In [23]:
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 29],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 10, 31, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 32],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 33],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 34],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 35,  2, 36],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 37],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 19],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 38,  2],
       [ 0,  0,  0,  0,  0,  0

#BUILD THE MODEL

In [24]:
#Architecture:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(len(labels), activation='softmax'))

In [25]:
#Compile:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [26]:
#summary:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 30)            1590      
_________________________________________________________________
bidirectional (Bidirectional (None, 20, 128)           48640     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 52,702
Trainable params: 52,702
Non-trainable params: 0
____________________________________________________

In [27]:
#Fit the model:
chatbot_model = model.fit(padded_sequences, np.array(training_labels),epochs=400, validation_split=0.0)

Epoch 1/400
2/2 [==============================] - 4s 26ms/step - loss: 2.0802 - accuracy: 0.0909
Epoch 2/400
2/2 [==============================] - 0s 19ms/step - loss: 2.0773 - accuracy: 0.0909
Epoch 3/400
2/2 [==============================] - 0s 17ms/step - loss: 2.0763 - accuracy: 0.0909
Epoch 4/400
2/2 [==============================] - 0s 18ms/step - loss: 2.0751 - accuracy: 0.0909
Epoch 5/400
2/2 [==============================] - 0s 18ms/step - loss: 2.0734 - accuracy: 0.0909
Epoch 6/400
2/2 [==============================] - 0s 17ms/step - loss: 2.0723 - accuracy: 0.0909
Epoch 7/400
2/2 [==============================] - 0s 17ms/step - loss: 2.0712 - accuracy: 0.0909
Epoch 8/400
2/2 [==============================] - 0s 19ms/step - loss: 2.0700 - accuracy: 0.0909
Epoch 9/400
2/2 [==============================] - 0s 16ms/step - loss: 2.0691 - accuracy: 0.0909
Epoch 10/400
2/2 [==============================] - 0s 17ms/step - loss: 2.0680 - accuracy: 0.0909
Epoch 11/400
2/2 [=

Let's save the trained network.

In [28]:
model.save('chatbot_model')

INFO:tensorflow:Assets written to: chatbot_model/assets


INFO:tensorflow:Assets written to: chatbot_model/assets


In [31]:
import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

#IMPLEMENTING THE CHAT FUNCTION:

In [32]:
import random
import pickle

In [3]:
import json
with open("/content/drive/MyDrive/PROJECTS/CHATBOT PROJECT/chatbot dataset.json") as file:
  data = json.load(file)

In [34]:
#define a chat function:

def chat():
  # load trained model
  model = keras.models.load_model('chatbot_model')
  # load tokenizer object
  with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
  # load label encoder object
  with open('label_encoder.pickle', 'rb') as enc:
    label_encoder = pickle.load(enc)

  #parameters
  maxlen=20
  while True:
    print( "User: " , end="")
    input_sentence = input()
    if input_sentence.lower() == "quit":
      break

    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([input_sentence]),truncating='post', maxlen=maxlen))
    tag = label_encoder.inverse_transform([np.argmax(result)])

    for intent in data['intents']:
      if intent['tag'] == tag:
        print( "ChatBot:" , np.random.choice(intent['responses']))

#LET US CHAT WITH CHATBOT

In [35]:
print("Start messaging with the bot (type quit to stop)!" )
chat()

Start messaging with the bot (type quit to stop)!
User: Hiiii
ChatBot: I'm krishnya!
User: how are you
ChatBot: I.m Krishna's chatbot, your bot assistant
User: your name
ChatBot: I'm krishnya!
User:  bye
ChatBot: I'm krishnya!
User: I have a complaint
ChatBot: Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: Thank you
ChatBot: Any time!
User: quit
